<a href="https://colab.research.google.com/github/WenjunYuAnny/AI_Project_Foreign_Whisper/blob/text-to-speech/Text_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.io.wavfile import write as write_wav

## Using the Bark model for text-to-speech

In [2]:
pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-fak6aps7
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-fak6aps7
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.3 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=25674

In [3]:
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE
import numpy as np

In [4]:
import os

## Choosing the small model for faster processing

In [5]:
os.environ["SUNO_USE_SMALL_MODELS"] = "1"

In [6]:
preload_models()

text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 171MB/s]


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Retrieve the translated scripts from MS-3

In [8]:
def read_srt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        original_script = file.readlines()
    return original_script

def text_from_srt(original_scripts):
    translated_script_text = []
    current_text = ""

    for line in original_scripts:
        line = line.strip()
        if line.isdigit():
            if current_text:
                translated_script_text.append(current_text)
            current_text = ""
        elif '-->' in line:
            continue
        elif line:
            if current_text != "":
                current_text += " "
            current_text += line
    if current_text:
        translated_script_text.append(current_text)
    return translated_script_text

In [9]:
source_dir = "/content/drive/MyDrive/2023 Fall Artificial Intelligence/Foreign Whisper/translated_srt"

In [10]:
output_dir = "/content/drive/MyDrive/2023 Fall Artificial Intelligence/Foreign Whisper/translated_audio"

## Producing the audio from the translated transcripts using a male French speaking voice

In [12]:
for filename in os.listdir(source_dir):
  text_path = os.path.join(source_dir, filename)
  sentences = text_from_srt(read_srt(text_path))
  SPEAKER = "v2/fr_speaker_0"
  silence = np.zeros(int(0.25 * SAMPLE_RATE))

  pieces = []
  for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]
  concatenated_audio = np.concatenate(pieces)
  output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".wav")
  write_wav(output_path, SAMPLE_RATE, concatenated_audio)

100%|██████████| 11/11 [00:06<00:00,  1.74it/s]
